In [1]:
# Install required packages in the current Jupyter kernel
# Uncomment the following lines if you need to install these libraries
# If you run into permission issues, try with the --user option
#import sys
#!pip install -q rdflib networkx matplotlib
#!{sys.executable} -m pip install rdflib networkx matplotlib pandas stringdb --user


In [2]:
%load_ext autoreload
%autoreload 2
import sampler
import grabhogs_sparql
import map2string_fast
import addfrombloom
import rdflib
import SPARQLWrapper
import colour
import itertools
import networkx as nx
import glob
from matplotlib import pyplot as plt

In [3]:
datapath = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/402676.protein.links.rdf.v11.5.txt.gz'
# RDF graph loading
rg = sampler.load_graph(datapath)

datapath2 = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.info.rdf.v11.5/402676.protein.info.rdf.v11.5.txt.gz'
# RDF graph loading
rg_info  = sampler.load_graph(datapath2)

rdflib Graph loaded successfully with 2620310 triples
rdflib Graph loaded successfully with 14559 triples


In [4]:
links = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/*.protein.links.rdf.v11.5.txt.gz'
linkfiles = glob.glob(links)
linkfiles = { l:{ 'links':l , 'info':l.replace('protein.links' , 'protein.info' ) } for l in linkfiles}
print(len(linkfiles ))

1831


In [5]:
subjs = rg.subjects( unique = True)
seed = next(subjs)
print(seed)

https://string-db.org/network/402676.B6K4T6


In [6]:
subg = sampler.sample( rg = rg , seed = seed,  layer_limit= 2 , sample_run = 20 )
print(set([p for p in subg.predicates()]))
print("rdflib Graph sampled successfully with {} triples".format(len(subg)))

176 704
{rdflib.term.URIRef('https://string-db.org/rdf/any-confidence'), rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff')}
rdflib Graph sampled successfully with 596 triples


In [7]:
subg = sampler.add_xrefs( rg_info , subg )
print("rdflib Graph annotated successfully with {} triples".format(len(subg)))
print(set([p for p in subg.predicates()]))
cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot")
i = 0 
for s,p,o in subg.triples((None, cross_ref, None)):
    print(s,p,o)
    i+= 1
    if i > 10:
        break

57
57
rdflib Graph annotated successfully with 653 triples
{rdflib.term.URIRef('https://string-db.org/rdf/any-confidence'), rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot'), rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff')}
https://string-db.org/network/402676.B6JVA8 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JVA8
https://string-db.org/network/402676.B6JV67 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JV67
https://string-db.org/network/402676.B6JUU0 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JUU0
https://string-db.org/network/402676.B6JWW4 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JWW4
https://string-db.org/network/402676.B6K038 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6K038
https://string-db.o

In [8]:
#proteins_by_species , results_subj_para , results_subj_ortho  = grabhogs_sparql.grab_hogs( subg , cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot") )

In [9]:
#print(results_subj_ortho)

In [10]:
orthograph =  grabhogs_sparql.grab_hogs_graph( subg , cross_ref , sparql_endpoint= None
            , USE_CASE = 1 , verbose = True , cross_ref_prop = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot"))

57
Total time for 114 SPARQL queries: 40.11890888214111 seconds (multiple batch calls in: 0 cases)
Num errors: 0


In [11]:
from rdflib import Graph, URIRef
print("rdflib Graph annotated successfully with {} triples".format(len(orthograph)))
#get all species
taxa = [ 'protein1_uniprot_taxon_orth' , 'protein1_uniprot_taxon_para']
species = set( [ o for t in taxa  for s,p,o in orthograph.triples((None, URIRef(t), None))  ])


rdflib Graph annotated successfully with 684998 triples


In [12]:
#get all the results for that species
taxa = [ 'orth_protein_uniprot_taxon_orth' , 'para_protein_uniprot_taxon_para']
#get all proteins for each species
prots_by_species = { spec: set([s  for s,p,o in orthograph.triples((None, URIRef(t), spec ))]) for t in taxa for spec in species  }
prots_by_species = { spec:prots_by_species[spec] for spec in prots_by_species if len(prots_by_species[spec])   }
prots_by_species = { spec:[ p.replace('https://string-db.org/network/' , '' ) for p in prots_by_species[spec] ] for spec in prots_by_species }

In [13]:
server = 'dna071'
ortho_xrefgraph = map2string_fast.mapall(prots_by_species , serverurl= "http://"+server+":3030/string_fuseki/sparql" , retgraph = True)

In [14]:
orthograph += ortho_xrefgraph
subg += orthograph
#we have interactions for one species and ortho info to all others

In [15]:
filters = addfrombloom.load_filters()

In [16]:
#lets add the interactions for all using the bloom filters
print( len(orthograph ) , set([p for p in orthograph.predicates() ]) )
#get string ids by species
pred = rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot')
interactions = []
for spec in prots_by_species:
    stringids = [ s for prot in prots_by_species[spec] for s,p,o in orthograph.triples((None, pred , prot )) ]
    #stringids = [ s.replace('https://string-db.org/network/' , '' ) for s in stringids ]
    if len(stringids ) > 2 :
        interactions += addfrombloom.check_allvall( objects = stringids , urlstring = 'https://string-db.org/network/' , filters = filters )
[subg.add(t) for t in interactions]

684998 {rdflib.term.URIRef('taxon_taxon_para'), rdflib.term.URIRef('protein1_orth_protein_uniprot'), rdflib.term.URIRef('taxon_para_protein_uniprot'), rdflib.term.URIRef('protein1_uniprot_orth_protein_uniprot'), rdflib.term.URIRef('protein1_uniprot_taxon_para'), rdflib.term.URIRef('protein1_taxon_para'), rdflib.term.URIRef('orth_protein_orth_protein_uniprot'), rdflib.term.URIRef('taxon_para_protein'), rdflib.term.URIRef('protein1_orth_protein'), rdflib.term.URIRef('protein1_uniprot_taxon_orth'), rdflib.term.URIRef('taxon_taxon_orth'), rdflib.term.URIRef('taxon_orth_protein_uniprot'), rdflib.term.URIRef('para_protein_taxon_para'), rdflib.term.URIRef('protein1_uniprot_taxon'), rdflib.term.URIRef('orth_protein_uniprot_taxon_orth'), rdflib.term.URIRef('protein1_taxon'), rdflib.term.URIRef('protein1_taxon_orth'), rdflib.term.URIRef('protein1_para_protein_uniprot'), rdflib.term.URIRef('orth_protein_taxon_orth'), rdflib.term.URIRef('protein1_protein1_uniprot'), rdflib.term.URIRef('para_protei

[]

In [17]:
#halelujah we have a graph with everything in it
#serialize to turtle format
v = subg.serialize(format="ttl")
with open('testgraph.ttl', 'w') as graphout:
    graphout.write(v)

In [18]:
print(len(subg))
readg = Graph()
readg.parse('testgraph.ttl')
print(len(readg))
#we can save the subgraphs in rdf...


1664371
1664371


In [93]:
# create a new heterodata object
from torch_geometric.data import HeteroData , Data
import torch_geometric.utils 
from rdflib import RDF
import numpy as np
import scipy.sparse
import torch



def sparse2pairs(sparsemat, matrows = None):
    '''
    This functino takes a sparse matrix and returns a list of pairs of the non zero entries
    args:
        sparsemat: a sparse matrix
        matrows: a list of the matrix rows to keep
    Returns:    
        a list of pairs of the non zero entries
    '''
    if matrows :
        sparsemat = sparsemat[matrows,:]
        sparsemat = sparsemat[:,matrows]
    sparsemat = scipy.sparse.find(sparsemat)
    return np.vstack([sparsemat[0],sparsemat[1]])


def test_heterograph(rdf_graph , verbose = True):
    data = HeteroData()
    # assign edge types from the predicate
    edge_types = set([p for s,p,o in rdf_graph])
    edge_types = {e:i for i,e in enumerate(edge_types)}

    uris = {}
    subtypes = {}
    for sub, pred, obj in rdf_graph:
        for o in (sub,obj):
            if isinstance(o, URIRef):
                uri_type = rdf_graph.qname(o).split(":")[0]  # get the namespace prefix
                if uri_type not in uris:
                    uris[uri_type] = []
                    subtypes[uri_type] = set([])
                uris[uri_type].append(o)
                subtypes[uri_type].add( ''.join(o.n3().split('/')[0:-1] ) )
    
    if verbose == True:
        for t in subtypes:
            print(subtypes[t])
            #compile the x data matrix for each subtype
            
    
    node_index_by_type = { uritype : { n:i for i,n in enumerate( set(uris[uritype]) ) } for uritype in uris }
    
    #todo add the diff types of uris within each namespace as 1hot encoded for x attribute of each node
    
    
    for t1,t2 in itertools.product(node_index_by_type,node_index_by_type):
            rows = node_index_by_type[t1]
            columns = node_index_by_type[t2]
            if verbose == True:
                print(t1,len(rows),t2, len(columns))
                
            for edge_type in edge_types:
                
                # create a dictionary of nodes
                
                triples =  [ (s,p,o) for s,p,o in rdf_graph.triples((None, edge_type, None))  if ( s in rows and o in columns )  ]
                
                if len(triples)>0:
                    
                    adj = scipy.sparse.lil_matrix((len(rows), len(columns)))
                    
                    for s,p,o in triples:    
                        adj[rows[s], columns[o]] = 1
                    
                    
                    if verbose == True:
                        print(adj.sum())
                        pass
                    data[ t1 , p.n3() , t2 ].edge_index = torch.tensor( sparse2pairs(adj) ,  dtype=torch.long )
                    #data[ t1 , p.n3() , t2 ].edge_index  = torch_geometric.utils.add_self_loops(data[ t1 , p.n3() , t2 ].edge_index , num_nodes = (len(rows), len(columns)) )
                    data[ t1 , p.n3() , t2 ].edge_index = torch_geometric.utils.to_undirected(data[ t1 , p.n3() , t2 ].edge_index  )
    return data
data = test_heterograph(subg)
print(data)

{'<https:omabrowser.orgomainfo'}
{'<http:purl.uniprot.orgtaxonomy'}
{'<http:purl.uniprot.orguniprot'}
{'<https:string-db.orgnetwork'}
ns5 51682 ns5 51682
54076.0
6069.0
ns5 51682 ns3 2449
3589.0
5676.0
59.0
39312.0
47221.0
ns5 51682 ns4 70079
72936.0
69189.0
7528.0
59.0
7372.0
ns5 51682 ns6 57
ns3 2449 ns5 51682
5676.0
47221.0
ns3 2449 ns3 2449
615.0
2441.0
ns3 2449 ns4 70079
7063.0
64366.0
ns3 2449 ns6 57
ns4 70079 ns5 51682
5704.0
49507.0
ns4 70079 ns3 2449
3311.0
35741.0
56.0
68717.0
7101.0
ns4 70079 ns4 70079
67302.0
7091.0
ns4 70079 ns6 57
ns6 57 ns5 51682
ns6 57 ns3 2449
ns6 57 ns4 70079
ns6 57 ns6 57
149.0
149.0
149.0
149.0
HeteroData(
  (ns5, <protein1_orth_protein>, ns5)={ edge_index=[2, 108152] },
  (ns5, <protein1_para_protein>, ns5)={ edge_index=[2, 12126] },
  (ns5, <protein1_taxon_para>, ns3)={ edge_index=[2, 7178] },
  (ns5, <para_protein_taxon_para>, ns3)={ edge_index=[2, 11352] },
  (ns5, <protein1_taxon>, ns3)={ edge_index=[2, 118] },
  (ns5, <protein1_taxon_orth>, ns

In [54]:
print((s,p,o))

import time
from rdflib import Graph, RDF, RDFS, OWL , Literal , BNode

def separate_graph(g):
    
    # Create empty dictionaries for the different types of objects
    uris = {}
    bnodes = {}
    literals = {}
    # Iterate over all triples in the graph
    for s, p, o in g:
        if isinstance(o, URIRef):
            uri_type = g.qname(o).split(":")[0]  # get the namespace prefix
            if uri_type not in uris:
                uris[uri_type] = []
            uris[uri_type].append(o)

        elif isinstance(o, BNode):
            if o not in bnodes:
                bnodes[o] = []

            # Iterate over all triples that have the BNode as a subject
            for s2, p2, o2 in g.triples((o, None, None)):
                bnodes[o].append((s2, p2, o2))

        elif isinstance(o, Literal):
            literal_type = None
            if RDF.type in g[o]:
                for t in g[o][RDF.type]:
                    if t in [RDFS.Literal, OWL.DatatypeProperty]:
                        continue
                    if isinstance(t, URIRef):
                        literal_type = g.qname(t).split(":")[0]  # get the namespace prefix
                        break

            if literal_type is None:
                literal_type = "unknown"

            if literal_type not in literals:
                literals[literal_type] = []
            literals[literal_type].append(o)

    # Print out the results
    print("URIs:")
    for uri_type, uris in uris.items():
        print(uri_type, len(uris))

    print("\nBlank nodes:")
    for bnode, triples in bnodes.items():
        print(bnode, len(triples))

    print("\nLiterals:")
    for literal_type, literals in literals.items():
        print(literal_type, len(literals))
t0 = time.time()
separate_graph(subg)
print(time.time()-t0)

def separate_graph_further(g):


    # Create empty dictionaries for the different types of objects
    uris = {}
    bnodes = {}
    literals = {}

    # Iterate over all triples in the graph
    for s, p, o in g:
        if isinstance(o, URIRef):
            uri_type = g.qname(o)
            if uri_type not in data:
                data[uri_type] = {}
            if RDF.type in g[o]:
                for t in g[o][RDF.type]:
                    if t in [RDFS.Resource, OWL.Thing]:
                        continue
                    if isinstance(t, URIRef):
                        type_name = g.qname(t)
                        if type_name not in data[uri_type]:
                            data[uri_type][type_name] = []
                        data[uri_type][type_name].append(o)
        else:
            if isinstance(o, Literal):
                literal_type = None
                if RDF.type in g[o]:
                    for t in g[o][RDF.type]:
                        if t in [RDFS.Literal, OWL.DatatypeProperty]:
                            continue
                        if isinstance(t, URIRef):
                            literal_type = g.qname(t)
                            break

                if literal_type is None:
                    literal_type = "unknown"

                if literal_type not in data:
                    data[literal_type] = []
                data[literal_type].append(o)

    # Print out the results
    for uri_type, types in data.items():
        print(f"Namespace: {uri_type}")
        for type_name, uris in types.items():
            print(f"  Type: {type_name}")
            print(f"    URIs: {len(uris)}")
        print("\n")

        
#too slow
t0 = time.time()
separate_graph_further(subg)
print(time.time()-t0)


(rdflib.term.URIRef('https://string-db.org/network/402676.B6JXG0'), rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot'), rdflib.term.Literal('http://purl.uniprot.org/uniprot/B6JXG0'))
URIs:
ns3 213839
ns4 302906
ns5 168253
ns6 596

Blank nodes:

Literals:
unknown 57
2.2281651496887207


KeyboardInterrupt: 

In [ ]:
G = rdflib.extras.external_graph_libs.rdflib_to_networkx_multidigraph( subg , lambda s, p, o: {'data':{'key':p  , 'weight':1} } )

In [ ]:
# example usage
uniprot_id = "http://purl.uniprot.org/uniprot/A0A3B5R6M3" 
serverurl = '
results =  map2string_fast.map2string_SPARQL(uniprot_id , serverurl = )
print(results)

In [ ]:
preds = [ p for p in rg.predicates( unique = True)]
red = colour.Color('red')
blue = colour.Color('blue')
c = [ c.hex_l for c in  list(red.range_to(blue, len(preds))) ]
colors = { p:c[i] for i,p in enumerate(preds)}
delta = 2/len(preds)
curve = { p:delta*i for i,p in enumerate(preds) }
style = itertools.cycle([ '-', '--' ])
line_style = { p:next(style) for p in preds }

In [ ]:
pos = nx.circular_layout( G )
f = plt.figure()
f.set_figwidth(15)
f.set_figheight(15)
plt.plot()
#plot the whole mess
ax = plt.gca()
for e in G.edges(data = True):
    ax.annotate("",
                xy=pos[e[0]], xycoords='data',
                xytext=pos[e[1]], textcoords='data',
                arrowprops=dict(arrowstyle="-", color=colors[e[2]['data']['key']],
                                shrinkA=5, shrinkB=5, lw = e[2]['data']['weight'], ls = line_style[e[2]['data']['key']],
                                patchA=None, patchB=None, alpha = e[2]['data']['weight'],
                                connectionstyle="arc3,rad=rrr".replace('rrr',str(0.3*curve[e[2]['data']['key']])
                                ),
                                ),
                )

nx.draw_networkx_nodes(G, pos, node_color = 'b', node_size = 150, alpha = .5)
labels=nx.draw_networkx_labels(G , pos = pos , font_size= 15 , font_color='w')

plt.axis('off')
plt.show()


In [ ]:
#map string neighbours to OMA entries

#jump a few steps in HOGs

#fish for STRING interactions in other species


